In [1]:
import os
os.chdir('../streamlit')


In [7]:
from utils import load_all_data

all_data = load_all_data(exclude_teg_50 = True, exclude_incomplete_tegs = True)

max_stab = max(all_data['Stableford'])

# stab_dist = all_data['Stableford'].value_counts().reset_index()
# stab_dist.columns = ['Stableford','Count']
# print(stab_dist)

all_data['Stableford'].value_counts().reset_index().sort_values(by='Stableford')

,Stableford,count
3,0.0,756
2,1.0,1153
0,2.0,2044
1,3.0,1707
4,4.0,340
5,5.0,30


In [2]:
from utils import calculate_handicaps_for_teg, load_all_data

all_data = load_all_data()

hc = calculate_handicaps_for_teg(target_teg_num = 18, all_data = all_data, include_incomplete = True)

print(hc)

2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-12 11:15:30.646 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-09-

           Player  Handicap  TEG1_Score  TEG2_Score  TEG1_Adjusted  \
0      Alex BAKER      36.0       33.75       31.75          36.25   
1    David MULLIN      20.0       37.00       35.50          20.00   
2  Gregg WILLIAMS      20.0       30.75       35.75          21.25   
3  John PATTERSON      28.0       33.75       36.00          28.25   
4       Jon BAKER      18.0       41.50       32.75          16.50   
5  Stuart NEUMANN      27.0       36.00       39.00          27.00   

   TEG2_Adjusted  TEG1_HC  TEG2_HC         Source  
0          34.25     34.0     30.0  handicaps.csv  
1          20.50     21.0     20.0  handicaps.csv  
2          16.25     16.0     16.0  handicaps.csv  
3          26.00     26.0     26.0  handicaps.csv  
4          22.25     22.0     19.0  handicaps.csv  
5          26.00     27.0     29.0  handicaps.csv  


In [3]:
import pandas as pd
import numpy as np
from decimal import Decimal, ROUND_HALF_UP

def calculate_handicaps_for_teg_new(
    target_teg_num: int,
    all_data: pd.DataFrame = None,
    include_incomplete: bool = False,
    debug: bool = False,
    debug_print: bool = False,
    rounding_method: str = "bankers",  # "bankers" (Python default) or "half_up"
) -> pd.DataFrame:
    """
    Calculate handicaps for a given TEG based on weighted average of previous two TEGs.

    Handicaps from handicaps.csv ALWAYS take precedence over calculated values.

    Handicaps are calculated using:
    - 75% weight from (target_teg_num - 1)
    - 25% weight from (target_teg_num - 2)
    - Adjusted gross score = 36 - average stableford per round + handicap
    - Players missing from a TEG are deemed to have scored 36 stableford points per round

    Returns:
        pd.DataFrame with at least:
        ['Player','Handicap','TEG1_Score','TEG2_Score','TEG1_Adjusted','TEG2_Adjusted',
         'TEG1_HC','TEG2_HC','Source']
        Plus debug-friendly extras when available:
        ['TEG1_Score_Total','TEG2_Score_Total','Calculated_Handicap_Unrounded']
        And a debug payload in df.attrs['debug'] if debug=True.
    """

    def _round_hc(x: float) -> int:
        if rounding_method == "bankers":
            return int(round(x))
        elif rounding_method == "half_up":
            return int(Decimal(x).quantize(Decimal('1'), rounding=ROUND_HALF_UP))
        else:
            raise ValueError("rounding_method must be 'bankers' or 'half_up'")

    # Load data if not provided
    if all_data is None:
        all_data = load_all_data(exclude_teg_50=True, exclude_incomplete_tegs=not include_incomplete)

    # Ensure we have the required TEGs for calculation
    teg1_num = target_teg_num - 1  # 75% weight
    teg2_num = target_teg_num - 2  # 25% weight
    if teg2_num < 15:
        raise ValueError(
            f"Cannot calculate handicaps for TEG {target_teg_num}: requires TEG 15 or later "
            f"(needs 2 previous TEGs with reliable data)."
        )

    # All players who have ever played (excluding Henry Meller, by your rule)
    all_players = sorted([p for p in all_data['Player'].unique() if p != 'Henry MELLER'])

    # Get handicaps from the handicaps.csv file
    try:
        handicaps_df = read_file(HANDICAPS_CSV)
        target_teg_col = f'TEG {target_teg_num}'
        teg1_col = f'TEG {teg1_num}'
        teg2_col = f'TEG {teg2_num}'

        # Code -> full name mapping for columns in handicaps.csv
        player_mapping = {
            'DM': 'David MULLIN',
            'GW': 'Gregg WILLIAMS',
            'JP': 'John PATTERSON',
            'JB': 'Jon BAKER',
            'SN': 'Stuart NEUMANN',
            'AB': 'Alex BAKER'
        }

        # Existing handicaps for the target TEG (take precedence)
        existing_handicaps = {}
        target_row = handicaps_df[handicaps_df['TEG'] == target_teg_col]
        if not target_row.empty:
            row = target_row.iloc[0]
            for code, full_name in player_mapping.items():
                if code in handicaps_df.columns and not pd.isna(row[code]) and row[code] != 0:
                    existing_handicaps[full_name] = float(row[code])

        # Previous TEG handicaps (used in adjusted gross)
        teg1_handicaps = {}
        teg2_handicaps = {}

        teg1_row = handicaps_df[handicaps_df['TEG'] == teg1_col]
        teg2_row = handicaps_df[handicaps_df['TEG'] == teg2_col]

        if not teg1_row.empty:
            row = teg1_row.iloc[0]
            for code, full_name in player_mapping.items():
                if code in handicaps_df.columns and not pd.isna(row[code]) and row[code] != 0:
                    teg1_handicaps[full_name] = float(row[code])

        if not teg2_row.empty:
            row = teg2_row.iloc[0]
            for code, full_name in player_mapping.items():
                if code in handicaps_df.columns and not pd.isna(row[code]) and row[code] != 0:
                    teg2_handicaps[full_name] = float(row[code])

    except Exception:
        # If handicaps file not available, use default handicaps
        try:
            import streamlit as st
            st.warning("Could not load handicaps file. Using default handicaps of 18 for all players.")
        except Exception:
            print("WARNING: Could not load handicaps file. Using default handicaps of 18 for all players.")
        existing_handicaps = {}
        teg1_handicaps = {player: 18 for player in all_players}
        teg2_handicaps = {player: 18 for player in all_players}

    # Slice TEG data
    teg1_data = all_data[all_data['TEGNum'] == teg1_num]
    teg2_data = all_data[all_data['TEGNum'] == teg2_num]
    teg0_data = all_data[all_data['TEGNum'] == target_teg_num]

    # Helper: stableford per player (mean & total per round inside TEG)
    def _stableford_summary(teg_df: pd.DataFrame, players: list) -> pd.DataFrame:
        if teg_df.empty:
            base = pd.DataFrame({'Player': players})
            base['TEG_Score'] = np.nan
            base['TEG_Score_Total'] = np.nan
            base['TEG_Rounds'] = 0
            return base

        # Stableford per round per player
        per_round = (teg_df.groupby(['Player', 'Round'], as_index=False)['Stableford']
                     .sum())  # sum across holes within a round

        stats = (per_round.groupby('Player')
                 .agg(TEG_Score=('Stableford', 'mean'),
                      TEG_Score_Total=('Stableford', 'sum'),
                      TEG_Rounds=('Round', 'nunique'))
                 .reindex(players))  # ensure all players represented

        stats = stats.reset_index()
        return stats

    # Compute summaries
    sb1 = _stableford_summary(teg1_data, all_players)
    sb2 = _stableford_summary(teg2_data, all_players)

    # Apply "36 per round if absent" rule for mean & total
    rounds_in_teg1 = int(teg1_data['Round'].nunique())
    rounds_in_teg2 = int(teg2_data['Round'].nunique())

    # For players with no rounds in the TEG, set mean=36 and total = 36 * rounds_in_that_teg
    for df, rcount in ((sb1, rounds_in_teg1), (sb2, rounds_in_teg2)):
        missing = (df['TEG_Rounds'].isna()) | (df['TEG_Rounds'] == 0)
        df.loc[missing, 'TEG_Score'] = 36.0
        df.loc[missing, 'TEG_Score_Total'] = 36.0 * float(rcount)
        df.loc[missing, 'TEG_Rounds'] = 0

    # Rename for clarity when merging
    sb1 = sb1.rename(columns={'TEG_Score': 'TEG1_Score',
                              'TEG_Score_Total': 'TEG1_Score_Total',
                              'TEG_Rounds': 'TEG1_Rounds'})
    sb2 = sb2.rename(columns={'TEG_Score': 'TEG2_Score',
                              'TEG_Score_Total': 'TEG2_Score_Total',
                              'TEG_Rounds': 'TEG2_Rounds'})

    # Previous handicaps (CSV -> dicts -> DataFrame; default 18 if missing)
    prev_hc = pd.DataFrame({
        'Player': all_players,
        'TEG1_HC': [teg1_handicaps.get(p, 18) for p in all_players],
        'TEG2_HC': [teg2_handicaps.get(p, 18) for p in all_players],
    })

    # Build one row per player with all the bits
    base = pd.DataFrame({'Player': all_players})
    debug_df = (base
                .merge(sb1, how='left', on='Player')
                .merge(sb2, how='left', on='Player')
                .merge(prev_hc, how='left', on='Player'))

    # Adjusted gross per TEG
    debug_df['TEG1_Adjusted'] = 36.0 - debug_df['TEG1_Score'] + debug_df['TEG1_HC']
    debug_df['TEG2_Adjusted'] = 36.0 - debug_df['TEG2_Score'] + debug_df['TEG2_HC']

    # Weighted handicap (unrounded, then rounded)
    debug_df['Calculated_Handicap_Unrounded'] = 0.75 * debug_df['TEG1_Adjusted'] + 0.25 * debug_df['TEG2_Adjusted']
    debug_df['Calculated_Handicap'] = debug_df['Calculated_Handicap_Unrounded'].apply(_round_hc)

    # Apply CSV overrides for target TEG
    debug_df['Existing_Handicap'] = debug_df['Player'].map(existing_handicaps).astype('float64')
    debug_df['Handicap'] = np.where(
        debug_df['Existing_Handicap'].notna(),
        debug_df['Existing_Handicap'],
        debug_df['Calculated_Handicap']
    ).astype(int)
    debug_df['Source'] = np.where(debug_df['Existing_Handicap'].notna(), 'handicaps.csv', 'calculated')

    # Produce the main result (keeps your original columns; adds totals + unrounded for visibility)
    result_cols = [
        'Player', 'Handicap',
        'TEG1_Score', 'TEG2_Score',
        'TEG1_Score_Total', 'TEG2_Score_Total',
        'TEG1_HC', 'TEG2_HC',
        'TEG1_Adjusted', 'TEG2_Adjusted',
        'Calculated_Handicap_Unrounded',
        'Source'
    ]
    result_df = debug_df[result_cols].copy()

    # Build the small "which tournaments are which" table
    def _label_from_df(df, fallback_num: int) -> str:
        # Prefer 'TEG' column if present, otherwise "TEG {num}"
        if not df.empty and 'TEG' in df.columns and df['TEG'].notna().any():
            # Use the most common value if multiple
            val = df['TEG'].mode(dropna=True)
            return val.iloc[0] if not val.empty else f"TEG {fallback_num}"
        return f"TEG {fallback_num}"

    teg_map = pd.DataFrame([
        {'Label': 'TEG-2', 'TEGNum': teg2_num, 'TEG': _label_from_df(teg2_data, teg2_num),
         'RoundsInTEG': rounds_in_teg2},
        {'Label': 'TEG-1', 'TEGNum': teg1_num, 'TEG': _label_from_df(teg1_data, teg1_num),
         'RoundsInTEG': rounds_in_teg1},
        {'Label': 'TEG',   'TEGNum': target_teg_num, 'TEG': _label_from_df(teg0_data, target_teg_num),
         'RoundsInTEG': int(teg0_data['Round'].nunique()) if not teg0_data.empty else np.nan},
    ])

    # Optional: attach debug payload & print
    if debug:
        # Compact tables for exactly what you asked to see
        stableford_means_totals = result_df[['Player', 'TEG1_Score', 'TEG1_Score_Total',
                                             'TEG2_Score', 'TEG2_Score_Total']].copy()
        prev_handicaps = result_df[['Player', 'TEG1_HC', 'TEG2_HC']].copy()
        adjusted_gross = result_df[['Player', 'TEG1_Adjusted', 'TEG2_Adjusted']].copy()
        final_handicaps = result_df[['Player', 'Handicap', 'Source']].copy()

        debug_payload = {
            'teg_map': teg_map.sort_values('Label'),
            'stableford_means_totals': stableford_means_totals.sort_values('Player'),
            'prev_handicaps': prev_handicaps.sort_values('Player'),
            'adjusted_gross': adjusted_gross.sort_values('Player'),
            'final_handicaps': final_handicaps.sort_values('Player'),
            'full_debug_table': debug_df.sort_values('Player')  # everything together
        }
        result_df.attrs['debug'] = debug_payload

        if debug_print:
            print("\n=== Which tournaments are TEG, TEG-1, TEG-2 ===")
            print(debug_payload['teg_map'].to_string(index=False))

            print("\n=== Mean & total Stableford (TEG-1 / TEG-2) ===")
            print(debug_payload['stableford_means_totals'].to_string(index=False))

            print("\n=== Previous handicaps used (TEG-1 / TEG-2) ===")
            print(debug_payload['prev_handicaps'].to_string(index=False))

            print("\n=== Adjusted gross (36 - mean + HC) ===")
            print(debug_payload['adjusted_gross'].to_string(index=False))

            print("\n=== Final handicap (CSV takes precedence) ===")
            print(debug_payload['final_handicaps'].to_string(index=False))

    return result_df


In [6]:
calculate_handicaps_for_teg_new(target_teg_num=18,all_data=all_data,include_incomplete=False,debug=True,debug_print=True)

2025-09-12 11:26:31.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 11:26:31.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 11:26:31.975 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.



=== Which tournaments are TEG, TEG-1, TEG-2 ===
Label  TEGNum    TEG  RoundsInTEG
  TEG      18 TEG 18          NaN
TEG-1      17 TEG 17          4.0
TEG-2      16 TEG 16          4.0

=== Mean & total Stableford (TEG-1 / TEG-2) ===
        Player  TEG1_Score  TEG1_Score_Total  TEG2_Score  TEG2_Score_Total
    Alex BAKER       33.75             135.0       31.75             127.0
  David MULLIN       37.00             148.0       35.50             142.0
Gregg WILLIAMS       30.75             123.0       35.75             143.0
John PATTERSON       33.75             135.0       36.00             144.0
     Jon BAKER       41.50             166.0       32.75             131.0
Stuart NEUMANN       36.00             144.0       39.00             156.0

=== Previous handicaps used (TEG-1 / TEG-2) ===
        Player  TEG1_HC  TEG2_HC
    Alex BAKER       18       18
  David MULLIN       18       18
Gregg WILLIAMS       18       18
John PATTERSON       18       18
     Jon BAKER       18    

,Player,Handicap,TEG1_Score,TEG2_Score,TEG1_Score_Total,TEG2_Score_Total,TEG1_HC,TEG2_HC,TEG1_Adjusted,TEG2_Adjusted,Calculated_Handicap_Unrounded,Source
0,Alex BAKER,21,33.75,31.75,135.0,127.0,18,18,20.25,22.25,20.7500,calculated
1,David MULLIN,17,37.00,35.50,148.0,142.0,18,18,17.00,18.50,17.3750,calculated
2,Gregg WILLIAMS,22,30.75,35.75,123.0,143.0,18,18,23.25,18.25,22.0000,calculated
3,John PATTERSON,20,33.75,36.00,135.0,144.0,18,18,20.25,18.00,19.6875,calculated
4,Jon BAKER,15,41.50,32.75,166.0,131.0,18,18,12.50,21.25,14.6875,calculated
5,Stuart NEUMANN,17,36.00,39.00,144.0,156.0,18,18,18.00,15.00,17.2500,calculated


In [ ]:
from utils import load_and_prepare_handicap_data, HANDICAPS_CSV, get_complete_teg_data, get_teg_data_inc_in_progress, get_number_of_completed_rounds_by_teg
import pandas as pd

hc = load_and_prepare_handicap_data(HANDICAPS_CSV)
hc['TEGNum'] = hc["TEG"].str[-2:].str.strip().astype(int)

all_data = load_all_data(  exclude_incomplete_tegs = False)
teg_data = get_teg_data_inc_in_progress()
num_rounds = get_number_of_completed_rounds_by_teg(all_data)

TEG_needed = 17
TEG_2 = TEG_needed - 2
TEG_1 = TEG_needed - 1
tegnums = [TEG_1, TEG_2]

hc_x = hc[hc['TEGNum'].isin(tegnums)]  # GET HANDICAP DATA FOR THE TWO TEGs WE'RE INTERESTED IN

stab_x = teg_data[teg_data['TEGNum'].isin(tegnums)][['Pl','Stableford','TEGNum']].copy() # GET STABLEFORD RESULTS FOR THE TWO TEGs WE'RE INTERESTED IN
stab_x = stab_x.merge(num_rounds, on="TEGNum", how="left")
stab_x['ave_stab'] = stab_x['Stableford'] / stab_x['num_rounds']
hc_merged = hc_x.merge(stab_x, on = ['Pl','TEGNum'], how = 'left') # COMBINE HC WITH STABLEFORD FOR PREVIOUS TEGS
hc_merged['ave_stab'] = pd.to_numeric(hc_merged['ave_stab'], errors="coerce").fillna(36) # SET STABLEFORD OF 144 (36 PER ROUND) FOR MISSING DATA
# hc_merged['Stableford'] = pd.to_numeric(hc_merged['Stableford'], errors="coerce").fillna(144) # SET STABLEFORD OF 144 (36 PER ROUND) FOR MISSING DATA
# NB NEXT LINE WOULD BE GOOD TO REPLACE '/4' WITH '/ NUMBER OF ROUNDS'
hc_merged['AdjGross'] = 36 - hc_merged['ave_stab'] + hc_merged['HC'] # CALCULATE THE ADJUSTED GROSS FOR HANDICAPPING


pivoted = hc_merged.pivot(index="Pl", columns="TEGNum", values="AdjGross") #PIVOT THE DATA TO BRING ADJGROSS INTO ROWS FOR THE WEIGHTED AVE CALC

result = (
    0.75 * pivoted[TEG_1] +
    0.25 * pivoted[TEG_2]
).reset_index(name="hc_raw") # CREATE THE UNROUNDED 'RAW' HANDICAP

result['hc'] = result['hc_raw'].round(0).astype(int) # CREATE THE ROUNDED HANDICAP
#print(result)

hc_output = result[['Pl','hc']]



   Pl  hc
0  AB  34
1  DM  21
2  GW  16
3  JB  22
4  JP  26
5  SN  27


In [6]:
import os
os.chdir('../streamlit')
from utils import get_incomplete_tegs, load_all_data
from utils import load_and_prepare_handicap_data, HANDICAPS_CSV, get_complete_teg_data, get_teg_data_inc_in_progress, get_number_of_completed_rounds_by_teg
import pandas as pd

all_data = load_all_data(exclude_incomplete_tegs = False)

print(len(get_incomplete_tegs(all_data)))


0


In [ ]:
import os
os.chdir('../streamlit')
from utils import get_teg_data_inc_in_progress, get_complete_teg_data, get_round_data, load_all_data, load_and_prepare_handicap_data, HANDICAPS_CSV


teg_data_inc_progress = get_teg_data_inc_in_progress()
teg_data_completed = get_complete_teg_data()

last_completed_teg = max(teg_data_completed['TEGNum'])
next_teg = last_completed_teg + 1
in_progress = (teg_data_inc_progress['TEGNum'] == next_teg).any()

print(last_completed_teg)
# print(last_completed_round_teg)
print(next_teg)
print(in_progress)


@st.cache_data
def get_next_teg_and_check_if_in_progress():
    """Get TEG completion status (cached for performance)."""
    teg_data_inc_progress = get_teg_data_inc_in_progress() 
    teg_data_completed = get_complete_teg_data()  
    
    last_completed_teg = max(teg_data_completed['TEGNum']) 
    next_teg = last_completed_teg + 1 
    in_progress = (teg_data_inc_progress['TEGNum'] == next_teg).any()
    
    return last_completed_teg, next_teg, in_progress



17
18
False


In [14]:
import os
os.chdir('../streamlit')
from utils import get_hc

print(get_hc(18))
get_hc(18).to_clipboard(index=False)

   Pl   hc_raw  hc
0  AB  35.7500  36
1  DM  20.1250  20
2  GW  20.0000  20
3  JB  17.9375  18
4  JP  27.6875  28
5  SN  26.7500  27
